[View in Colaboratory](https://colab.research.google.com/github/aminzabardast/Tensorflow-Tutorials/blob/dev/6_Saving_Tensor_Structure_And_Variables.ipynb)

# Saving Tensor Structure and Variables

Developed model in Notebook 5

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


def conv2d(input_data, kernel_shape, activation_function=tf.nn.relu, name=None):
    weights = tf.Variable(tf.truncated_normal(kernel_shape, stddev=0.1))
    biases = tf.Variable(tf.constant(.1, shape=[kernel_shape[3]]))
    conv_result = tf.nn.conv2d(input_data, weights, strides=[1, 1, 1, 1], padding='SAME', name=name) + biases
    return activation_function(conv_result)


def maxPool2x2(input_data, name=None):
    return tf.nn.max_pool(input_data, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)


def fullyConnected(input_data, shape, activation_function=tf.nn.relu, name=None):
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    biases = tf.Variable(tf.zeros([1, shape[1]]) + 0.1)
    return activation_function(tf.matmul(input_data, weights) + biases, name=name)


def computeAccuracy(v_xs, v_ys):
    y_pre = sess.run(rfc2, feed_dict={xs: v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result


with tf.name_scope('Input'):
    # Intensity values are between 0 and 255.
    # This will be converted to between 0 and 1.
    xs = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='Data')/255
    # MNIST data are storred in vectors and the need to be reshaped to 28*28 image.
    x_images = tf.reshape(xs,shape=[-1, 28, 28, 1], name='ReshapedData')
    
with tf.name_scope('Truth'):
    ys = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='Truth')

with tf.name_scope('Conv1'):
    # 5x5 Kernel, Feature Map Input 1 (Gray scale image) and Output 32
    kernel1_shape = [5, 5, 1, 32]
    # Using kernel1 to calculate the convolved layer / Output size: 28 x 28 x 32
    rconv1 = conv2d(x_images, kernel1_shape)

with tf.name_scope('MaxPool1'):
    # Max polling the result of Conv1 layer / Output size: 14 x 14 x 32
    rpool1 = maxPool2x2(rconv1)

with tf.name_scope('Conv2'):
    # 3x3 Kernel, Feature Map Input 32 and Output 64
    kernel2_shape = [3, 3, 32, 64]
    # Using kernel2 to calculate the convolved layer / Output size: 14 x 14 x 64
    rconv2 = conv2d(rpool1, kernel2_shape)

with tf.name_scope('MaxPool2'):
    # Max polling the result of Conv2 layer / Output size: 7 x 7 x 64
    rpool2 = maxPool2x2(rconv2)

with tf.name_scope('FC1'):
    # Shape of the layer
    fc1_shape = [7*7*64, 1024]
    # Flattening From [n,7,7,64] to [n,3136]
    rpool2_flattened = tf.reshape(rpool2, shape=[-1, 7*7*64])
    rfc1 = fullyConnected(rpool2_flattened, fc1_shape)

with tf.name_scope('FC2'):
    # Shape of the layer
    fc2_shape = [1024, 10]
    rfc2 = fullyConnected(rfc1, fc2_shape, tf.nn.softmax, name='prediction')

with tf.name_scope('Loss'):
    # Cross Entropy as the loss function
    crossEntropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(rfc2), reduction_indices=[1]))
    tf.summary.scalar(name='CrossEntropy', tensor=crossEntropy)

with tf.name_scope('Optimizer'):
    # Optmizing using Simple Gradient Descent
    trainStep = tf.train.GradientDescentOptimizer(learning_rate=0.01, name='train_step').minimize(crossEntropy)

# Creating Session
sess = tf.Session()

# Initiating Variable
init = tf.global_variables_initializer()

# Merging all summaries
merged = tf.summary.merge_all()

# Creating File Writers
trainWriter = tf.summary.FileWriter(logdir='logs/train', graph=sess.graph)
testWriter = tf.summary.FileWriter(logdir='logs/test', graph=sess.graph)

# Initiating Session
sess.run(init)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Training and Saving

In [2]:
saver = tf.train.Saver(max_to_keep=4, keep_checkpoint_every_n_hours=2)

# Training
for epoch in range(100):
    # Training with some Images in one epoch
    trainBatchXs, trainBatchYs = mnist.train.next_batch(300)
    testBatchXs, testBatchYs = mnist.test.next_batch(300)
    
    # Forward and backward pass
    sess.run(trainStep, feed_dict={xs: trainBatchXs, ys: trainBatchYs})
    
    # Adding the state of network to logs
    trainWriter.add_summary(sess.run(merged, feed_dict={xs: trainBatchXs, ys: trainBatchYs}), epoch)
    testWriter.add_summary(sess.run(merged, feed_dict={xs: testBatchXs, ys: testBatchYs}), epoch)
    
    if (epoch+1) % 10 == 0 or epoch == 0:
        print('Epoch: {}'.format(epoch+1))
    
    if (epoch+1) % 10 == 0:
        saver.save(sess, 'saved-network/model', global_step=epoch+1)

# Calculating final accuracy
accuracy = computeAccuracy(mnist.test.images, mnist.test.labels)*100
print('\nFinal Accuracy: %{0:.2f}'.format(accuracy))

Epoch: 1
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Epoch: 100

Final Accuracy: %88.23


In [3]:
!tar czvf saved.tar.gz saved-network

from google.colab import files
files.download('saved.tar.gz')

!rm -rvf logs* saved*

saved-network/
saved-network/model-80.data-00000-of-00001
saved-network/model-100.index
saved-network/model-100.meta
saved-network/model-90.index
saved-network/model-70.meta
saved-network/model-90.data-00000-of-00001
saved-network/model-100.data-00000-of-00001
saved-network/model-70.data-00000-of-00001
saved-network/model-80.index
saved-network/model-90.meta
saved-network/checkpoint
saved-network/model-70.index
saved-network/model-80.meta
removed 'logs/train/events.out.tfevents.1533506288.18e7c463b108'
removed directory 'logs/train'
removed 'logs/test/events.out.tfevents.1533506288.18e7c463b108'
removed directory 'logs/test'
removed directory 'logs'
removed 'saved-network/model-80.data-00000-of-00001'
removed 'saved-network/model-100.index'
removed 'saved-network/model-100.meta'
removed 'saved-network/model-90.index'
removed 'saved-network/model-70.meta'
removed 'saved-network/model-90.data-00000-of-00001'
removed 'saved-network/model-100.data-00000-of-00001'
removed 'saved-network/mod